In [ ]:
!pip install pyTelegramBotAPI
!pip3 -q install youtube_transcript_api
!pip3 -q install transformers

In [ ]:
####################################################################################
# impor libraries

import os
import psutil
import telebot
import string
import torch
import numpy as np
from datetime import datetime


from youtube_transcript_api import YouTubeTranscriptApi
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
####################################################################################
# model for summary
model_name = 'facebook/bart-large-cnn' # can be any model which satisfies quality and inference constraints

tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)#.to(device)

In [ ]:
####################################################################################
# tg bot

token = '5565148517:AAG3LoJUgMAmymBg6BFq8ARLJ-bc26fcySQ'
bot = telebot.TeleBot(token)

def summary(message, threshold):
    link = message.text
    unique_id = link[threshold:]

    bot.send_message(message.chat.id, "Отлично, это действительно ссылка на YouTuBe видео, дай мне, пожалуйста, немного времени и я скоро отправлю саммари)")

    # count time
    start = datetime.now()

    # make summary part
    subtitles_by_intervals = YouTubeTranscriptApi.get_transcript(unique_id, languages=['en', 'ru']) 
    subtitles = " ".join([x['text'] for x in subtitles_by_intervals])
    subtitles = subtitles#.to(device)

    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, framework='pt') # will be used in audio section as well
    # tokenizer_kwargs = {'truncation': True, 'max_length': 512}

    result = summarizer(
                subtitles,
                # tokenizer params
                # max_length=1024, truncation=True,
                # model params
                # min_length=120, max_length=240, 
                # num_beams=4, length_penalty=2.0, early_stopping=True,
                truncation=True, return_text=True, return_tensors=False
            )[0]['summary_text']

    bot.send_message(message.chat.id, "Саммари готово)")
    bot.send_message(message.chat.id, f"{result}")

    end = datetime.now()
    bot.send_message(message.chat.id, f"Мне потребовалось {np.round((end - start).total_seconds(), 2)} секунд для ответа)")

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет!) \nЯ умею делать саммари по видео с YouTuBe. Отправь мне ссылку на видео, и я сделаю по нему саммари)')

@bot.message_handler(content_types=['text'])
def action_function(message):
    
    # if youtube link has the "brauser" form like https://www.youtube.com/watch?v= we preprocess one way
    if (message.text[:32] == "https://www.youtube.com/watch?v="):
        summary(message, threshold = 32)
        
    # if youtube link has the "iphone" form like https://youtu.be/ we preprocess another way
    elif (message.text[:17] == "https://youtu.be/"):
        summary(message, threshold = 17)

    else:
        bot.send_message(message.chat.id, "Проверь, пожалуйста, что ссылка корректна, не могу найти видео на YouTuBe с такой ссылкой)")


bot.polling(none_stop=True)

In [ ]:
# text_1 = "https://youtu.be/Z1RJmh_OqeA"
# text_2 = "https://www.youtube.com/watch?v=Z1RJmh_OqeA"